# **`utils_clean.py`**

This notebook contains the functions called by the main_clean.py script.

In [ ]:
from djitellopy import Tello  
import cv2  
import numpy as np

| Code Example                | Code Explanation                                                      |
|-----------------------------|------------------------------------------------------------------------|
| `from djitellopy import Tello`  | Import the `Tello` class from the `djitellopy` library for drone control. |
| `import cv2`                  | Import OpenCV (`cv2`), a library for computer vision tasks.                 |
| `import numpy as np`          | Import the `numpy` library for numerical operations and alias it as `np`.  |


In [ ]:
def initializeTello():
    """Initialize Tello drone and return the object.

    param: None

    return: Tello object
    """
    myDrone = Tello()
    myDrone.connect()
    myDrone.for_back_velocity = myDrone.left_right_velocity = myDrone.up_down_velocity = myDrone.yaw_velocity = myDrone.speed = 0
    print(f"Drone Battery: {myDrone.get_battery()}%")
    myDrone.streamoff()
    myDrone.streamon()
    return myDrone

| Code Example                            | Code Explanation                                                               |
|-----------------------------------------|---------------------------------------------------------------------------------|
| `def initializeTello():`                | Define a function named `initializeTello` with no parameters.                   |
| `"""Initialize Tello drone and ...`     | A docstring providing a brief description of the function's purpose and its return type.  |
| `myDrone = Tello()`                     | Create a new Tello drone object and store it in the variable `myDrone`.         |
| `myDrone.connect()`                     | Establish a connection between the program and the Tello drone.                 |
| `myDrone.for_back_velocity = ... = 0`   | Initialize all drone velocities and speed to zero.                              |
| `print(f"Drone Battery: ...")`          | Print the current battery level of the drone to the console.                    |
| `myDrone.streamoff()`                   | Turn off the video stream for the drone.                                       |
| `myDrone.streamon()`                    | Turn on the video stream for the drone.                                        |
| `return myDrone`                        | Return the initialized Tello drone object.                                      |


In [ ]:
def telloGetFrame(myDrone, w=360, h=240):
    """Capture and return a resized video frame from Tello drone.

    param: myDrone: Tello object
    param: w: frame width
    param: h: frame height

    return: resized video frame
    """

    frame = myDrone.get_frame_read().frame
    img = cv2.resize(frame, (w, h))
    return img

| Code Example                            | Code Explanation                                                                                   |
|-----------------------------------------|-----------------------------------------------------------------------------------------------------|
| `def telloGetFrame(myDrone, w=360, h=240):` | Define a function named `telloGetFrame` that takes a Tello object, frame width `w`, and frame height `h` as parameters. Width and height have default values of 360 and 240, respectively. |
| `"""Capture and return ...`             | A docstring that provides a brief description of the function, its parameters, and its return type.  |
| `frame = myDrone.get_frame_read().frame` | Retrieve the current video frame from the Tello drone and store it in the variable `frame`.          |
| `img = cv2.resize(frame, (w, h))`       | Use OpenCV's `resize` function to resize the captured frame to the dimensions specified by `w` and `h`. Store the resized frame in the variable `img`.  |
| `return img`                            | Return the resized video frame.                                                                      |


In [ ]:
def findFace(img):
    """Detect and return largest face coordinates and area.

    param: img: video frame

    return: img: frame with rectangle around largest face
    return: [cx, cy]: center of largest face
    return: area: area of largest face
    """

    # Detect face
    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.1, 6)
    
    # Get the largest face
    myFaceListC, myFaceListArea = [], []
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cx, cy, area = x + w // 2, y + h // 2, w * h
        myFaceListC.append([cx, cy])
        myFaceListArea.append(area)
    
    # Return center and area of the largest face
    if myFaceListArea:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceListC[i], myFaceListArea[i]]
    return img, [[0, 0], 0]

| Code Example                                     | Code Explanation                                                                       |
|--------------------------------------------------|----------------------------------------------------------------------------------------|
| `def findFace(img):`                             | Function definition; takes a video frame as input.                                      |
| `faceCascade = cv2.CascadeClassifier(...)`       | Load OpenCV's Haar cascade classifier for frontal face detection.                       |
| `imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`| Convert the frame to grayscale for easier face detection.                               |
| `faces = faceCascade.detectMultiScale(...)`      | Detect faces in the image and store their coordinates in a list.                       |
| `myFaceListC, myFaceListArea = [], []`           | Initialize empty lists to store center coordinates and areas of detected faces.        |
| `for (x, y, w, h) in faces:`                     | Loop through each detected face in the frame.                                          |
| `cv2.rectangle(img, (x, y), (x+w, y+h), ...)`    | Draw a rectangle around the detected face in the frame.                                |
| `cx, cy, area = x + w // 2, y + h // 2, w * h`   | Calculate the center coordinates and area of the current face.                          |
| `myFaceListC.append([cx, cy])`                   | Append the center coordinates to `myFaceListC`.                                        |
| `myFaceListArea.append(area)`                    | Append the area of the current face to `myFaceListArea`.                               |
| `if myFaceListArea:`                              | Check if any faces were detected.                                                       |
| `i = myFaceListArea.index(max(myFaceListArea))`  | Find the index of the face with the largest area.                                      |
| `return img, [myFaceListC[i], myFaceListArea[i]]`| Return the frame with the largest face highlighted, its center, and its area.           |
| `return img, [[0, 0], 0]`                         | If no faces were detected, return the frame as-is, along with a default center and area.|


In [ ]:
def findArea(img):
    """ Detect and return area of the largest face.

    param: img: video frame
    
    return: area: area of the largest face
    """
    
    # Reusing findFace for getting the area
    _, info = findFace(img)
    return info[1] if info else 0

| Code Example                        | Code Explanation                                                                           |
|-------------------------------------|--------------------------------------------------------------------------------------------|
| `def findArea(img):`                | Function definition; takes a video frame as input and returns the area of the largest face. |
| `_, info = findFace(img)`           | Call the previously defined `findFace()` function to get face information.                  |
| `return info[1] if info else 0`     | Return the area of the largest face; if no face is found, return 0.                        |


In [ ]:
def trackFace(myDrone, info, w, pid_yaw, pid_fb, pError_yaw, pError_fb, facearea, h, pError_ud, pid_ud, center, desiredfacearea):
    """Track face and adjust drone position.

    return: pError_yaw: previous error for yaw
    return: pError_fb: previous error for forward-backward
    return: pError_ud: previous error for up-down
    """

    # if no face detected, return previous errors
    if len(info) < 2:
        return pError_yaw, pError_fb, pError_ud
    
    error_yaw = info[0][0] - center                                             # error for yaw
    speed_yaw = pid_yaw[0] * error_yaw + pid_yaw[1] * (error_yaw - pError_yaw)  # PD values for yaw
    speed_yaw = int(np.clip(speed_yaw, -60, 60))                                # clip the speed
    
    error_fb = facearea - desiredfacearea                                       # error for forward-backward
    speed_fb = pid_fb[0] * error_fb + pid_fb[1] * (error_fb - pError_fb)        # PD values for forward-backward
    speed_fb = int(np.clip(speed_fb, -20, 20))                                  # clip the speed

    # PID values for up-down
    error_ud = info[0][1] - h // 2                                              # error for up-down
    speed_ud = pid_ud[0] * error_ud + pid_ud[1] * (error_ud - pError_ud)        # PD values for up-down
    speed_ud = int(np.clip(speed_ud, -25, 25))                                  # clip the speed    
    
    # Update previous errors
    print(speed_yaw, speed_fb, speed_ud)
    print(center, desiredfacearea)
    
    # Update previous errors
    if info[0][0]:
        myDrone.left_right_velocity = speed_yaw if abs(speed_yaw) < 20 else 0
        myDrone.yaw_velocity = speed_yaw if abs(speed_yaw) >= 20 else 0
        myDrone.for_back_velocity = -speed_fb
        myDrone.up_down_velocity = -speed_ud
    else:
        myDrone.for_back_velocity = myDrone.left_right_velocity = myDrone.up_down_velocity = myDrone.yaw_velocity = 0

    # Update previous errors
    if myDrone.send_rc_control:
        myDrone.send_rc_control(myDrone.left_right_velocity, myDrone.for_back_velocity, myDrone.up_down_velocity, myDrone.yaw_velocity)
    
    # Return previous errors
    return error_yaw, error_fb, error_ud

| Code Example                                 | Code Explanation                                                                          |
|----------------------------------------------|-------------------------------------------------------------------------------------------|
| `def trackFace(...)`                          | Function definition with multiple parameters including the drone object, PID constants, previous errors, and face area information. |
| `if len(info) < 2:`                           | Checks if a face is detected; if not, the function returns previous errors.                |
| `return pError_yaw, pError_fb, pError_ud`     | Returns the previous errors for yaw, forward-backward, and up-down when no face is detected. |
| `error_yaw = info[0][0] - center`             | Calculates the yaw error based on the face's center and the frame's center.                |
| `speed_yaw = pid_yaw[0] * error_yaw + pid_yaw[1] * (error_yaw - pError_yaw)` | Calculates the yaw speed adjustment using PID control. |
| `speed_yaw = int(np.clip(speed_yaw, -60, 60))`| Clips the yaw speed within the range [-60, 60].                                            |
| `error_fb = facearea - desiredfacearea`       | Calculates the forward-backward error based on current and desired face area.              |
| `speed_fb = pid_fb[0] * error_fb + pid_fb[1] * (error_fb - pError_fb)`  | Calculates the forward-backward speed adjustment using PD control.                        |
| `speed_fb = int(np.clip(speed_fb, -20, 20))`  | Clips the forward-backward speed within the range [-20, 20].                               |
| `error_ud = info[0][1] - h // 2`              | Calculates the up-down error based on the face's center and frame's height.                |
| `speed_ud = pid_ud[0] * error_ud + pid_ud[1] * (error_ud - pError_ud)`  | Calculates the up-down speed adjustment using PID control.                                |
| `speed_ud = int(np.clip(speed_ud, -25, 25))`  | Clips the up-down speed within the range [-25, 25].                                        |
| `if info[0][0]:`                              | Checks if a face is detected and adjusts drone's speed parameters accordingly.             |
| `if myDrone.send_rc_control:`                 | Sends the updated drone control commands if the send function is available.                |
| `return error_yaw, error_fb, error_ud`        | Returns the current errors for yaw, forward-backward, and up-down as the previous errors for the next iteration. |

